In [6]:
import uvicorn
from fastapi import FastAPI, Request, Response, Form, Depends, UploadFile, File, HTTPException, status
from fastapi.logger import logger
from fastapi.security import OAuth2PasswordBearer
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, concatenate, Flatten
from tensorflow.keras.models import load_model
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
import psycopg2
from numpy import save
from numpy import load
# import pickle5 as pickle
from json import loads, dumps
import jwt
import boto3
from sklearn.cluster import KMeans
import joblib
import math


def initiate_dbaccess():
    DB_NAME = "property_wallet_staging"
    DB_USER = "postgres"
    DB_PASS = "Panasonic1"
    DB_HOST = "karachihills.cquxw3hgishy.ap-south-1.rds.amazonaws.com"
    DB_PORT = 5432

    conn = psycopg2.connect(
        database=DB_NAME,
                            user=DB_USER,
                            password=DB_PASS,
                            host=DB_HOST,
                            port=DB_PORT)

    return conn


# print("Welcome ",Auth['email'])
conn=initiate_dbaccess()
print("Database connected successfully")

# result_dataframes = []

UserID = 503
print(UserID)

# Fetching distinct recommendations
cur = conn.cursor()
cur.execute("""SELECT IR."inventoryId", IR."type" FROM inventory_recomendation as IR WHERE
                IR."userId" = (%s) ORDER BY IR."createdAt" """, (UserID,))

distinct = cur.fetchall()
colnames = [desc[0] for desc in cur.description]
dis = pd.DataFrame(distinct, columns=colnames)
df_no_duplicates = dis[~dis.duplicated(keep='last')]
rows = df_no_duplicates.tail(4).values
# print("rows :")
# print(rows)

listing_ids = []
hotlisting_ids = []

for row in rows:
    inventory_id, inventory_type = row
    if inventory_type.lower() == 'listing':
        listing_ids.append(inventory_id)
    elif inventory_type.lower() == 'hotlisting':
        hotlisting_ids.append(inventory_id)

# print("listing_ids: ", listing_ids)
# print("hotlisting_ids: ", hotlisting_ids)


cur = conn.cursor()
cur.execute("""
    SELECT listing_invs.*, LA."title" as land_title FROM land_area as LA JOIN
    (SELECT inv_list.*, pro."city", pro."address" FROM project as pro JOIN
    (SELECT invents.*, list."saleCommission" FROM listing as list JOIN
    (SELECT invent.*, pst."title" as sub_category FROM project_sub_type as pst JOIN
    (SELECT inv."id", inv."projectSubTypeId", inv."projectId", inv."landAreaId",
    inv."price", inv."washRooms", inv."bedRooms", inv."landSize",
    pt."title" as category FROM inventory as inv JOIN 
    project_type as pt ON inv."projectTypeId"=pt."id") as invent
    ON pst."id"=invent."projectSubTypeId") as invents
    ON list."inventoryId"=invents."id" WHERE list."status"=(%s) 
    AND list."deletedAt" IS NULL) as inv_list
    ON inv_list."projectId"=pro."id") as listing_invs
    ON listing_invs."landAreaId"=LA."id" AND listing_invs."id" IN %s
    """, ('Approved', tuple(listing_ids)))
row1 = cur.fetchall()

# colnames = [desc[0] for desc in cur.description]
# listing_inventory = pd.DataFrame(row1, columns=colnames)
# result_dataframes.append(listing_inventory)

cur = conn.cursor()
cur.execute("""
SELECT listing_invs.*, LA."title" as land_title 
FROM land_area as LA 
JOIN (SELECT inv."landSize", pt."title" as category 
      FROM inventory as inv 
      JOIN project_type as pt ON inv."projectTypeId" = pt."id") as invent
ON pst."id" = invent."projectSubTypeId") as invents
ON hotlist."inventoryId" = invents."id" 
JOIN (SELECT inv."projectId", inv."landAreaId", inv."id"
      FROM inventory as inv 
      JOIN hotlist ON inv."id" = hotlist."inventoryId"
      WHERE hotlist."status" = 'Approved' AND hotlist."deletedAt" IS NULL) as inv_list
ON inv_list."projectId" = pro."id") as listing_invs
ON listing_invs."landAreaId" = LA."id" AND listing_invs."id" IN %s;
""", (8))
row2 = cur.fetchall()
# print("Row",row2)


colnames = [desc[0] for desc in cur.description]
row=row1+row2
result_dataframes = pd.DataFrame(row, columns=colnames)
# print(hotlisting_inventory)
# result_dataframes.append(hotlisting_inventory)

result_dataframes.head(5)

Database connected successfully
503


TypeError: 'int' object does not support indexing

In [10]:
import sqlite3
import pandas as pd

# Assuming conn is your SQLite connection object

cur = conn.cursor()
cur.execute("""
    SELECT listing_invs.*, LA."title" as land_title 
    FROM land_area as LA 
    JOIN (
        SELECT inv."landSize", pt."title" as category 
        FROM inventory as inv 
        JOIN project_type as pt ON inv."projectTypeId" = pt."id"
    ) as invents ON invents."id" = inv_list."inventoryId" 
    JOIN (
        SELECT inv."projectId", inv."landAreaId", inv."id"
        FROM inventory as inv 
        JOIN hotlist ON inv."id" = hotlist."inventoryId"
        WHERE hotlist."status" = (%s) AND hotlist."deletedAt" IS NULL
    ) as inv_list ON inv_list."projectId" = listing_invs."id"
    JOIN listing_invs ON listing_invs."landAreaId" = LA."id" AND listing_invs."id" IN %s
    """, ('Approved', tuple(hotlisting_ids)))

rows = cur.fetchall()
colnames = [desc[0] for desc in cur.description]

result_dataframes = pd.DataFrame(rows, columns=colnames)
print(result_dataframes)


SyntaxError: syntax error at or near ")"
LINE 15: ...ting_invs."landAreaId" = LA."id" AND listing_invs."id" IN ()
                                                                       ^


In [6]:
import uvicorn
from fastapi import FastAPI, Request, Response, Form, Depends, UploadFile, File, HTTPException, status
from fastapi.logger import logger
from fastapi.security import OAuth2PasswordBearer
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, concatenate, Flatten
from tensorflow.keras.models import load_model
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
import psycopg2
from numpy import save
from numpy import load
# import pickle5 as pickle
from json import loads, dumps
import jwt
import boto3
from sklearn.cluster import KMeans
import joblib
import math


def initiate_dbaccess():
    DB_NAME = "property_wallet_staging"
    DB_USER = "postgres"
    DB_PASS = "Panasonic1"
    DB_HOST = "karachihills.cquxw3hgishy.ap-south-1.rds.amazonaws.com"
    DB_PORT = 5432

    conn = psycopg2.connect(
        database=DB_NAME,
                            user=DB_USER,
                            password=DB_PASS,
                            host=DB_HOST,
                            port=DB_PORT)

    return conn

UserID = 8

# print("Welcome ",Auth['email'])
conn=initiate_dbaccess()
print("Database connected successfully")

cur = conn.cursor()
cur.execute("""SELECT IR."inventoryId",IR."type" FROM inventory_recomendation as IR WHERE
                IR."userId" = (%s) ORDER BY IR."createdAt" """,(UserID,))

distinct = cur.fetchall()
colnames = [desc[0] for desc in cur.description]
dis = pd.DataFrame(distinct, columns = colnames)
df_no_duplicates = dis[~dis.duplicated(keep='last')]
rows = df_no_duplicates.tail(4).values

inv_id=df_no_duplicates["inventoryId"].tail(4)

listing_ids = []
hotlisting_ids = []

df=pd.DataFrame(rows,columns=colnames)
df["type"]=[typ.lower() for typ in df["type"].values]
df1=df.groupby('type')['inventoryId'].apply(list).reset_index(name="inventories")

row1,row2 = [],[]

if "listing" in df1["type"].values:
    listing_ids=df1[df1["type"]=="listing"]['inventories'].values[0]

    cur = conn.cursor()
    cur.execute("""
        SELECT listing_invs.*, LA."title" FROM land_area as LA JOIN
        (SELECT inv_list.*, pro."city", pro."address" FROM project as pro JOIN
        (SELECT invents.*, list."saleCommission" FROM listing as list JOIN
        (SELECT invent.*, pst."title" FROM project_sub_type as pst JOIN
        (SELECT inv."id", inv."projectSubTypeId", inv."projectId", inv."landAreaId",
        inv."price", inv."washRooms", inv."bedRooms", inv."landSize",
        pt."title" FROM inventory as inv JOIN
        project_type as pt ON inv."projectTypeId"=pt."id") as invent
        ON pst."id"=invent."projectSubTypeId") as invents
        ON list."inventoryId"=invents."id" WHERE list."status"=(%s)
        AND list."deletedAt" IS NULL) as inv_list
        ON inv_list."projectId"=pro."id") as listing_invs
        ON listing_invs."landAreaId"=LA."id" AND listing_invs."id" IN %s
        """, ('Approved', tuple(listing_ids)))
    row1 = cur.fetchall()

if "hotlisting" in df1["type"].values:
    hotlisting_ids=df1[df1["type"]=="hotlisting"]['inventories'].values[0]
    cur = conn.cursor()
    cur.execute("""
        SELECT listing_invs.*, LA."title" FROM land_area as LA JOIN
        (SELECT inv_list.*, pro."city", pro."address" FROM project as pro JOIN
        (SELECT invents.*, hotlist."saleCommission" FROM hot_listing as hotlist JOIN
        (SELECT invent.*, pst."title" FROM project_sub_type as pst JOIN
        (SELECT inv."id", inv."projectSubTypeId", inv."projectId", inv."landAreaId",
        inv."price", inv."washRooms", inv."bedRooms", inv."landSize",
        pt."title" FROM inventory as inv JOIN
        project_type as pt ON inv."projectTypeId"=pt."id") as invent
        ON pst."id"=invent."projectSubTypeId") as invents
        ON hotlist."inventoryId"=invents."id" WHERE hotlist."status"=(%s)
        AND hotlist."deletedAt" IS NULL) as inv_list
        ON inv_list."projectId"=pro."id") as listing_invs
        ON listing_invs."landAreaId"=LA."id" AND listing_invs."id" IN %s
        """, ('Approved', tuple(hotlisting_ids)))
    row2 = cur.fetchall()

print("listing_ids: ", listing_ids)
print("hotlisting_ids: ", hotlisting_ids)

colnames = [desc[0] for desc in cur.description]
row=row1+row2
data = pd.DataFrame(row , columns = colnames)
data

Database connected successfully
listing_ids:  [118]
hotlisting_ids:  [126, 140, 139]


id  projectSubTypeId  projectId  landAreaId   price  washRooms  bedRooms  \
0  126                 9        178           1  350000        NaN       NaN   
1  140                 1        192           2  150000        2.0       3.0   

   landSize  title      title  saleCommission              city  \
0       260  Plots  Plot Form               0  Karachi Division   
1       500  Homes      House               0  Karachi Division   

         address         title  
0       orangi 1   Square Feet  
1  malir karachi  Square Yards